In [ ]:
#default_exp renderer

In [ ]:
#hide
%pip install svgwrite
%pip install webcolors

In [ ]:
#|export
from tetris.piece import Piece
from tetris.playfield import Playfield
from IPython.display import SVG
import svgwrite, colorsys, webcolors

class Renderer:
    def darken(self, hex_rgb: str, amount: float) -> str:
        rgb = webcolors.hex_to_rgb(hex_rgb)
        hls = colorsys.rgb_to_hls(rgb[0]/255, rgb[1]/255, rgb[2]/255)
        darkened_rgb = colorsys.hls_to_rgb(hls[0], max(0, min(1, amount * hls[1])), hls[2])
        return webcolors.rgb_to_hex(tuple([int(el * 255) for el in darkened_rgb]))

    def playfield_to_svg(self, pf: Playfield) -> SVG:
        A = pf.toarray()
        side_len = 10
        dwg = svgwrite.Drawing()
        dwg.viewbox(0,0,300,250)
        
        height = max(20, min(len(A), max(pf.col_height)))
        width = len(A[0])

        # border around entire playfield
        #dwg.add(dwg.rect((0, 0), (width*side_len, height*side_len), stroke="green"))

        for row in range(height):
            for col in range(len(A[0])):
                left_top = (col*side_len, (height-row)*side_len)
                width_height = (side_len, side_len)
                stroke = "#cccccc"
                fill = 'white'
                corner_radius = 0
                if A[row][col]:
                    piece_num = A[row][col]
                    base_fill = '#d1eaf3'
                    fill = self.darken(base_fill, 1 - (piece_num-1) * 0.2)
                    stroke = '#112543'
                    corner_radius = 1
                dwg.add(dwg.rect(left_top, width_height, stroke=stroke, fill=fill, rx=corner_radius, ry=corner_radius))
        return SVG(dwg.tostring())
 
    def piece_to_svg(self, piece: Piece) -> SVG:
        side_len = 10
        dwg = svgwrite.Drawing()
        dwg.viewbox(0,0,300,50)

        A = piece.toarray()
        height = len(A)
        width = len(A[0])
        
        for row in range(height):
            for col in range(width):
                left_top = (col*side_len, (height-row)*side_len)
                width_height = (side_len, side_len)
                fill = 'blue' if A[row][col] else 'white'
                stroke = '#112543'
                dwg.add(dwg.rect(left_top, width_height, fill=fill, stroke=stroke))
        return SVG(dwg.tostring())

# Pieces are uniquely colored

In [ ]:
from tetris.playfield import Playfield
from tetris.piece_loader import PieceLoader

pf = Playfield()
ldr = PieceLoader()
pf.add(ldr['j'], 0)
pf.add(ldr['l'], 2)
A = pf.toarray()
from tetris.renderer import Renderer
rndr = Renderer()
svg = rndr.playfield_to_svg(pf)
assert(svg.data.count('rect fill="#d1eaf3"') == 4)
assert(svg.data.count('rect fill="#89c9e0"') == 4)